In [1]:
%matplotlib inline
%run ../../import_envs.py
print('probtorch:', probtorch.__version__, 
      'torch:', torch.__version__, 
      'cuda:', torch.cuda.is_available())

probtorch: 0.0+5a2c637 torch: 0.4.1 cuda: True


In [2]:
from ag_ep import *

from local_enc import *
from global_oneshot import *
from global_enc_v1 import *
## if reparameterize continuous variables
Reparameterized = False
## Load dataset
data_path = "../gmm_dataset_3c"
Data = torch.from_numpy(np.load(data_path + '/obs.npy')).float()

NUM_DATASETS, N, D = Data.shape
K = 3 ## number of clusters
SAMPLE_SIZE = 2
NUM_HIDDEN_LOCAL = 32

MCMC_SIZE = 10
BATCH_SIZE = 20
NUM_EPOCHS = 200
LEARNING_RATE = 5 * 1e-4
CUDA = torch.cuda.is_available()

DEVICE = torch.device('cuda:0')
Model_Params = (N, K, D, MCMC_SIZE)

In [ ]:

PATH = 'ag-2samples'
Train_Params = (NUM_EPOCHS, NUM_DATASETS, SAMPLE_SIZE, BATCH_SIZE, CUDA, DEVICE, PATH)
# initialization
enc_z = Enc_z(K, D, NUM_HIDDEN_LOCAL, CUDA, DEVICE)
enc_eta = Enc_eta(K, D, CUDA, DEVICE, Reparameterized)
oneshot_eta = Oneshot_eta(K, D, CUDA, DEVICE, Reparameterized)
if CUDA:
    enc_z.cuda().to(DEVICE)
    enc_eta.cuda().to(DEVICE)
    oneshot_eta.cuda().to(DEVICE)
optimizer =  torch.optim.Adam(list(oneshot_eta.parameters())+list(enc_eta.parameters())+list(enc_z.parameters()),lr=LEARNING_RATE, betas=(0.9, 0.99))

models = (oneshot_eta, enc_eta, enc_z)
train(models, EUBO_init_eta, optimizer, Data, Model_Params, Train_Params)
torch.save(enc_z.state_dict(), "../weights/enc-z-%s" % PATH)
torch.save(enc_eta.state_dict(), "../weights/enc-eta-%s" % PATH)
torch.save(oneshot_eta.state_dict(), "../weights/oneshot-eta-%s" % PATH)
